In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO('yolov8l.pt')

In [ ]:
results = model.train(data='MediumModel2-3/data.yaml', epochs = 300, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.70 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.102 🚀 Python-3.12.3 torch-2.4.0.post301 CUDA:0 (NVIDIA L40, 45495MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=MediumModel2-3/data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save

100%|██████████| 6.25M/6.25M [00:00<00:00, 77.3MB/s]


AMP: checks passed ✅


train: Scanning /home/hawga970/Desktop/Brands/MediumModel2-3/train/labels.cache... 2887 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2887/2887 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 84, len(boxes) = 4308. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/home/hawga970/miniforge3/envs/detr_env/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /home/hawga970/Desktop/Brands/MediumModel2-3/valid/labels.cache... 272 images, 0 backgrounds, 0 corrupt: 100%|██████████| 272/272 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2, len(boxes) = 402. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/300      9.76G      1.413      5.127      1.687         62        640:  15%|█▍        | 27/181 [00:05<00:27,  5.58it/s]

In [ ]:
model = YOLO('runs/detect/train2/weights/last.pt')
model.train(resume=True)

In [3]:
model = YOLO('runs/detect/train/weights/best.pt')

In [4]:
result = model('test.png')
for r in result:
    r.save()
    


image 1/1 /home/hawga970/Desktop/Brands/test.png: 640x352 3 McDonaldss, 213.3ms
Speed: 1.9ms preprocess, 213.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 352)


In [6]:
import os
from PIL import Image
from sklearn.metrics import confusion_matrix
import numpy as np

def get_image_files(folder_path):
    image_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                image_files.append(os.path.join(root, file))
    return image_files

#CocaCola
y_true = np.array([1] * 28 + [0] * 30)
tpr = []
fpr = []
recall = []
precision = []

In [6]:
import re
conf = [0.00, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9]  
for con in conf:
    y_pred = []
    pred = 0
    actual = 0
    inc = 0
    image_files = get_image_files('VALID/Coke') + get_image_files('VALID/Null')
    for image in image_files:
        match = re.search(r'\d+', image)
        if match:
            P = int(match.group())
            actual = actual + P
        else: P = 0

        results = model(image, conf = con)
        for r in results:
            res = []
            nums = r.boxes.cls.cpu().numpy()
            names = r.names
            for i in nums:
                res.append(names[i])
        if "CocaCola" in str(res):
            y_pred.append(1)
        else: y_pred.append(0)

        TP = sum("CocaCola" in s for s in res)
        if TP>10 & P>10:
            TP=P
        if TP>P:
            FP = TP-P
            TP = P
            inc = inc+FP
        pred = pred+TP
        print("Confidence"+str(con))
        print("True Positives"+str(pred))
        print("False negatives"+str(inc))
        print("All predictions"+str(pred+inc))
    
    recall.append(pred/actual)
    precision.append(pred/(pred+inc))
    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    print(cm)
    tpr.append(cm[0,0] / sum(cm[0,:]))
    fpr.append(cm[1,0] / sum(cm[1,:]))


image 1/1 /home/hawga970/Desktop/Yolo8-SD/VALID/Coke/2Coke-ImgU.jpg: 640x448 48 CocaCola-Bottles, 4 CocaCola-Cans, 99 CocaCola-Logo-News, 5 CocaCola-Logo-Olds, 22 Fanta-Bottles, 1 Fanta-Can, 1 Fanta-Logo-Blue, 1 Fanta-Logo-White, 5 Pepsi-Bottles, 5 Pepsi-Logo-News, 5 Pepsi-Logo-Olds, 16 Sprite-Bottle-Greens, 17 Sprite-Bottle-Whites, 26 Sprite-Can-News, 24 Sprite-Logo-News, 21 Sprite-Logo-Olds, 391.3ms
Speed: 26.5ms preprocess, 391.3ms inference, 42.7ms postprocess per image at shape (1, 3, 640, 448)
Confidence0.0
True Positives2
False negatives154
All predictions156

image 1/1 /home/hawga970/Desktop/Yolo8-SD/VALID/Coke/1Coke-ImgP.png: 640x320 119 CocaCola-Bottles, 34 CocaCola-Cans, 61 CocaCola-Logo-News, 1 CocaCola-Logo-Old, 19 Fanta-Bottles, 7 Fanta-Cans, 1 Fanta-Logo-Blue, 2 Fanta-Logo-Whites, 12 Pepsi-Bottles, 2 Pepsi-Cans, 6 Pepsi-Logo-News, 4 Pepsi-Logo-Olds, 1 Sprite-Bottle-Green, 22 Sprite-Bottle-Whites, 2 Sprite-Logo-News, 7 Sprite-Logo-Olds, 229.1ms
Speed: 1.3ms preprocess, 2

KeyboardInterrupt: 

In [8]:
#image_files = get_image_files('test.png') #+ get_image_files('VALID/Null')
results = model('test.png', conf = 0.05)
count = 0
for r in results:
    name = "output/result_"+str(count)+".jpg"
    r.save(filename = str(name))
    count = count+1


image 1/1 /home/hawga970/Desktop/Brands/test.png: 640x352 3 McDonaldss, 211.8ms
Speed: 1.2ms preprocess, 211.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 352)


In [125]:
fpr

[1.0,
 0.06666666666666667,
 0.06666666666666667,
 0.03333333333333333,
 0.03333333333333333,
 0.03333333333333333,
 0.03333333333333333,
 0.03333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [126]:
tpr

[1.0,
 0.896551724137931,
 0.896551724137931,
 0.896551724137931,
 0.896551724137931,
 0.896551724137931,
 0.8620689655172413,
 0.8620689655172413,
 0.8620689655172413,
 0.8620689655172413,
 0.8620689655172413,
 0.8620689655172413,
 0.8620689655172413,
 0.8275862068965517,
 0.8275862068965517,
 0.7931034482758621,
 0.7931034482758621,
 0.7931034482758621,
 0.5517241379310345]

In [140]:
precision

[0.013225371120107961,
 0.8035714285714286,
 0.8301886792452831,
 0.8367346938775511,
 0.851063829787234,
 0.8863636363636364,
 0.8863636363636364,
 0.8863636363636364,
 0.8863636363636364,
 0.8837209302325582,
 0.8837209302325582,
 0.8837209302325582,
 0.9047619047619048,
 0.9024390243902439,
 0.9024390243902439,
 0.9,
 0.9230769230769231,
 1.0,
 1.0]

In [122]:
recall

[1.0,
 0.8367346938775511,
 0.8163265306122449,
 0.8163265306122449,
 0.7959183673469388,
 0.7959183673469388,
 0.7755102040816326,
 0.7551020408163265,
 0.7346938775510204,
 0.7142857142857143,
 0.6938775510204082,
 0.673469387755102,
 0.6530612244897959,
 0.6530612244897959,
 0.6530612244897959,
 0.6122448979591837,
 0.5714285714285714,
 0.46938775510204084,
 0.32653061224489793]